![](../images/rivacon_frontmark_combined_header.png)

In [ ]:
#
import pyvacon
import datetime as dt

import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools


import matplotlib.pyplot as plt
%matplotlib inline


#%matplotlib qt
#%matplotlib notebook

In [ ]:
refdate = dt.datetime(2017,2,20)
pyvacon.analytics.setLogLevel('DEBUG')
pyvacon.analytics.registerSerialization('depp')

# Callable Bonds
This section shows some examples pricing callable bonds using the CallableBondPdePricer. This pricer values a bond using a short rate model to model the interest rate (either the risk free or the interest rate modeling the underlying floating rate).
The specification used within the pricer is the CallableBondSpecification which allows for:
- Coupons payments at specified payment dates (for each coupon date a specific coupon size can be set)
- The coupons may be either fixed or float. The floating coupons are computed using reference floating coupon periods (the period is used to determine the floating rate). Each floating coupon can also have an individual spread added to the coupon.
- The bond may be called by the issuer at certain specified call dates at pre-specified call prices (note that currently no accrued interest is added to the call price)

### Create the necessary market data and setup pricing data
For simplicity we just create one artificial discount curve using the analyticsTestData module and use this curve as fixing and discount curve.

In [ ]:
pricing_data = pyvacon.pricing.CallableBondPdePricingData()
pricing_data.pricer = "CallableBondPdePricer"
pricing_data.pricingRequest = pyvacon.pricing.PricingRequest()
pricing_data.recovery = mkt_testdata.Credit.create_recovery(refdate, enums.SecuritizationLevel.SUBORDINATED, 
                                                               'RECOVERY_TEST')
pricing_data.riskfreeCurve = mkt_testdata.InterestRate.Curves.EONIA(refdate) # get artificial test curve from th test market data module

#downgrade_prob = 0
#upgrade_prob = 0.0
#pricing_data.transition = analyticsTestData.mkt.Credit.create_transition_matrix(refdate, 'TR_TEST', downgrade_prob,upgrade_prob)

pricing_data.transition = mkt_testdata.Credit.transtion_matrix(refdate,'Moodys95')

pricing_data.issuerRating = pyvacon.marketdata.Rating('TEST', refdate, 'A')

surv_date = [refdate + dt.timedelta(days=30*i) for i in range(10*12)]
pricing_data.sc = pricing_data.transition.computeSurvivalCurve(refdate, pricing_data.issuerRating, 'SC_TEST')

#analyticsTestData.mkt.Credit.create_survival_curve(refdate, 0.05, 'SC_TEST')
pricing_data.model = pyvacon.models.CIRModel('CIR_TEST',refdate, 1.0,0.01,0.1,0.001)
pricing_data.valDate = refdate
pricing_data.param = pyvacon.pricing.CallableBondPdePricingParameter()
pricing_data.param.model = 'CIR'
pricing_data.param.nSpotSteps = pricing_data.param.nSpotSteps 
pricing_data.param.nTimeStepsPerYear = pricing_data.param.nTimeStepsPerYear

### Setup specifications

In [ ]:
specifications = {} #dictionary containing the following different specifications

# floating periods
    
floating_periods = [refdate + dt.timedelta(days=i*365) for i  in range(11)]
floating_spreads = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
floating_rate_caps = [0.015, 0.015, 0.015, 0.015, 0.015, 0.015,0.015, 0.015, 0.015]
floating_rate_floors = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


# call dates
call_dates = [refdate + dt.timedelta(days=2+365 + i*365) for i in range(10)]
call_prices = [1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01]
    
# fixed coupons
fixed_coupon_dates = floating_periods[1:len(floating_periods)-1]
fixed_coupons = [0.02] * len(fixed_coupon_dates)
    
expiry = floating_periods[-1]
issue_date = refdate + dt.timedelta(days=2-365)

#### Fixed non-callable bond
10 yrs to maturity, yearly fixed coupons, non-callable

In [ ]:
floating_periods = []
floating_spreads = []
call_dates = []
call_prices = []
floating_rate_caps = []
floating_rate_floors = []
specifications['NONCALL_FIXED'] = pyvacon.instruments.CallableBondSpecification('NONCALL_FIXED', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)

#### Fixed callable bond
10 yrs to maturity, yearly fixed coupons, callable

In [ ]:

floating_periods = []
floating_spreads = []
call_dates = [refdate + dt.timedelta(days=2+365 + i*365) for i in range(10)]
call_prices = [1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01]
floating_rate_caps = []
floating_rate_floors = []

specifications['CALL_FIXED'] = pyvacon.instruments.CallableBondSpecification('CALL_FIXED', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)

#### Floating non-callable bond
10 yrs to maturity, non-callable, yearly floating coupons

In [ ]:
issue_date = refdate + dt.timedelta(days=2-365)

fixed_coupon_dates = []
fixed_coupon_values = []

floating_periods = [refdate + dt.timedelta(days=i*365) for i  in range(11)]
floating_spreads = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
floating_rate_caps = []
floating_rate_floors = []


call_dates = []
call_prices = []
floating_rate_caps = []
floating_rate_floors = []

specifications['NONCALL_FLOAT'] = pyvacon.instruments.CallableBondSpecification('NONCALL_FLOAT', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)

#### Floating callable bond
10 yrs to maturity, yearly floating coupons with call rights at fixing dates

In [ ]:
fixed_coupon_dates = []
fixed_coupon_values = []


floating_periods = [refdate + dt.timedelta(days=i*365) for i  in range(11)]
floating_spreads = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

call_dates = [refdate + dt.timedelta(days=2+365 + i*365) for i in range(10)]
call_prices = [1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01]

floating_rate_caps = []
floating_rate_floors = []
specifications['CALL_FLOAT'] = pyvacon.instruments.CallableBondSpecification('CALL_FLOAT', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)

for spread in [0.0, 0.0025, 0.005, 0.0075, 0.01, 0.015, 0.0125]:
    tmp = floating_spreads = [spread]*len(floating_spreads)
    specifications['CALL_FLOAT_' + str(spread)] = pyvacon.instruments.CallableBondSpecification('CALL_FLOAT', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)


#### Floating non-callable bond with caps
10 yrs to maturity, non-callable, yearly floating coupons capped at 1.5%

In [ ]:
issue_date = refdate + dt.timedelta(days=2-365)

fixed_coupon_dates = []
fixed_coupon_values = []
floating_periods = [refdate + dt.timedelta(days=i*365) for i  in range(11)]
floating_spreads = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

call_dates = []
call_prices = []
floating_rate_caps = [0.015, 0.015, 0.015, 0.015, 0.015, 0.015,0.015, 0.015, 0.015]
floating_rate_floors = []

specifications['NONCALL_CAP_FLOAT'] = pyvacon.instruments.CallableBondSpecification('NONCALL_CAP_FLOAT', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)

#### Floating callable bond with caps
10 yrs to maturity, non-callable, yearly floating coupons capped at 1.5%

In [ ]:
issue_date = refdate + dt.timedelta(days=2-365)

fixed_coupon_dates = []
fixed_coupon_values = []

floating_periods = [refdate + dt.timedelta(days=i*365) for i  in range(11)]
floating_spreads = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

call_dates = [refdate + dt.timedelta(days=2+365 + i*365) for i in range(10)]
call_prices = [1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01, 1.01]
floating_rate_caps = [0.015, 0.015, 0.015, 0.015, 0.015, 0.015,0.015, 0.015, 0.015]
floating_rate_floors = []

specifications['CALL_CAP_FLOAT'] = pyvacon.instruments.CallableBondSpecification('CALL_CAP_FLOAT', 'TEST_ISSUER', enums.SecuritizationLevel.SENIOR_UNSECURED, 'EUR',
                                             expiry, issue_date, 1.0, enums.DayCounter.ACT365_FIXED, fixed_coupon_dates,
                                             fixed_coupons, 'UDL_TEST', floating_periods, call_dates, 
                                              call_prices, floating_spreads, floating_rate_caps, floating_rate_floors)


### Pricing

In [ ]:
pricing_data.param.includeTransition = False
for key, spec in specifications.items():
    pricing_data.spec = spec
    #pricing_data.save( spec.getObjectId() +'_pricingdata.json', pricing_data)
    tic = dt.datetime.now()
    pr = pyvacon.pricing.price(pricing_data)
    print(key + ' price: ' + str(pr.getPrice()) + ' runtime: {}'.format(dt.datetime.now() - tic))

In [ ]:
pricing_data.param.includeTransition = True
for key, spec in specifications.items():
    pricing_data.spec = spec
    # pricing_data.save( spec.getObjectId() +'_pricingdata.json', pricing_data)
    tic = dt.datetime.now()
    pr = pyvacon.pricing.price(pricing_data)
    print(key + ' price: ' + str(pr.getPrice()) + ' runtime: {}'.format(dt.datetime.now() - tic))

### Plot market data

#### The transition matrix

In [ ]:
mkt_plot.transition_matrix_heatmap(pricing_data.transition, 3.0)
mkt_plot.plt.figure()
mkt_plot.transition_matrix_pd(pricing_data.transition, ['AAA', 'AA', 'A'])

#### The market risk free curve and the risk free curve induced by the model

In [ ]:
dates =  [refdate+ dt.timedelta(days=1), refdate + dt.timedelta(days=30), refdate + dt.timedelta(days=60), refdate + dt.timedelta(days=180),
          refdate + dt.timedelta(days=365), refdate + dt.timedelta(days=2*365), refdate + dt.timedelta(days=3*365), 
          refdate + dt.timedelta(days=5*365), refdate + dt.timedelta(days=10*365)]
cir_dc = model_tools.compute_yieldcurve(pricing_data.model, refdate,dates)
mkt_plot.curve(cir_dc, dates, refdate, True)
mkt_plot.curve(pricing_data.riskfreeCurve, dates, refdate, True)

#### Recovery (time dependent)

In [ ]:
mkt_plot.curve(pricing_data.recovery, dates, refdate)

### Projections

In [ ]:
cir_rate = [0.0025, 0.005, 0.0075, 0.01, 0.015, 0.02, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045]
projection = {}
#help(analytics.CIRModel)
for key in specifications.keys():
    projection[key] = []
for rate in cir_rate:
    pricing_data.model = pyvacon.models.CIRModel('CIR_TEST',refdate, 1.0,0.01,0.1,rate)
    for key, value in specifications.items():
        pricing_data.spec = value
        pr = pyvacon.pricing.price(pricing_data)
        projection[key].append(pr.getPrice())
plt.figure()
for key, value in projection.items():
    plt.plot(cir_rate, value, '-x', label=key)
#plt.legend()

In [ ]:
cir_vol = [0.1] #, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
projection = {}
#help(analytics.CIRModel)
for key in specifications.keys():
    projection[key] = []
for vol in cir_vol:
    pricing_data.model = pyvacon.analytics.CIRModel('CIR_TEST',refdate, 1.0,0.01,vol,0.02)
    for key, value in specifications.items():
        pricing_data.spec = value
        pr = pyvacon.analytics.price(pricing_data)
        projection[key].append(pr.getPrice())
plt.figure()
for key, value in projection.items():
    plt.plot(cir_vol, value, '-x', label=key)
#plt.legend()

# TODOS
- getPdeCoefficients für HullWhiteModel implementieren
- getPdeBoundaryConditions für HullWhiteModel implementieren
- Hull-White erweitern auf zeitabhaengiges sigma und a
- Hull-White Kalibrierung an Swaptions (Thomas Streuer fragen, ob das erledigt ist)
